In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv

In [2]:
## csv 파일 만들기 

samples = 1000
test_samples = 100
train_dataset = 'data/train_dataset.csv'
test_dataset = 'data/test_dataset.csv'

def write_dataset(samples, test_samples, train_dir, test_dir):
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]

    data = []
    label = []
    for i in range(samples):
        data.append(up)
        data.append(down)
        label.append([1])
        label.append([0])
        
    with open(train_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(samples-test_samples):
            writer.writerow(label[i] + data[i])
        print('train data is written')

    with open(test_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(test_samples):
            writer.writerow(label[i] + data[i])
        print('test data is written')
        
write_dataset(1000, 100, train_dataset, test_dataset)

train data is written
test data is written


In [24]:
import pandas as pd

data1 = pd.read_csv('data/train_dataset.csv')
data1[1:2]

,1,0,1.1,2,3,4,5,6,7,8,9
1,1,0,1,2,3,4,5,6,7,8,9


In [4]:
## 데이터셋을 읽어오기 (batch 사이즈를 정해줄수 있음, 셔플도 가능함)
train_dataset = tf.contrib.data.TextLineDataset(train_dataset).batch(32)#.shuffle(7777)
train_dataset = train_dataset.repeat(100)
## 셔플해도 상관없음

test_dataset = tf.contrib.data.TextLineDataset(test_dataset).batch(32)
test_dataset = test_dataset.repeat(99999999)
## 경로 적어주고 경로에 있는 데이터를 받아서 가져다줌 ## batch 사이즈 32개 

train_itr = train_dataset.make_one_shot_iterator() 
test_itr = test_dataset.make_one_shot_iterator() 
## iterator만들기: dataset 클레스도 내부에서 queue runners가 동작
## iterator는 그 queue에 dequeue를 해주는 역할
## batch가 실행될때마다 다음 batch 얻어옴 

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()## 다음 batch 얻어옴 


train_batch = tf.decode_csv(train_batch, [[0]]*11) 
test_batch = tf.decode_csv(test_batch, [[0]]*11)
## tf.expand_dims(array, axis) - 차원추가 


## decode를 안하면 dataset에서 reader로 TextLineDateset 메소드로 읽어왔기때문에 string type으로 출력됨 
## decode_csv를 하면 숫자로 읽어올수 있음
## Convert CSV records to tensors. Each column maps to one tensor
## [[0]]*11 = record defaults
## default를 integer 형, 값은 [0]으로 함 column들의 기본 형식을 정해준다 생각하면됨
## 예 :record_defaults = [ ["null"],[1],[1900],["null"],["null"]]

Instructions for updating:
Use `tf.data.TextLineDataset`.


In [15]:
[[0]]*11 ## 11 columns: integer 형식, default 값 = 0

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]

ERROR! Session/line number was not unique in database. History logging moved to new session 613


In [5]:

train_label  = tf.reshape(train_batch[0],[-1,1])  
train_label = tf.cast(train_label, tf.float32) 
## cast:정수형이면 tensorflow에서 실행되지 않아서 float로 바꿔줌 
## reshape 함수에 사용된 -1은 나누어 떨어지는 숫자를 자동으로 적용하겠다는 뜻이다. 
## 예: 12개의 요소가 있을 때, [3,-1]이라는 것은 [3,4]와 같고, [2,2,-1]이라는 것은 [2,2,3]과 같다. 
## 행과 열, 페이지에서 중심이 되는 것을 지정한 다음에, 나머지를 계산하기 싫을 때 주로 사용된다.

test_label  = tf.reshape(test_batch[0],[-1,1]) ## 899 rows *1 columns (배치사이즈에 맞게 조절됨 )
test_label = tf.cast(test_label, tf.float32)


# row_feature = tf.stack(decoded_batch[1:], axis = 0)
train_feature = tf.stack(train_batch[1:], axis = 1)
train_feature = tf.cast(train_feature, tf.float32)

test_feature = tf.stack(test_batch[1:], axis = 1)
test_feature = tf.cast(test_feature, tf.float32) ## float 형으로 바꿔줘야함 
# cast : intiger이면 tensorflow에서 실행되지 않아서 float로 바꿔줌 
#feature  = decoded_batch[1:] ## 1번째 이후 나머지 줄 

In [16]:
import numpy as np
np.array([[0],[1],[2],[3],[4],[5]]).reshape([-1,2]) 
## -1: 정해지지 않은 행을 뜻함, 정해진 열 두개에 대해 임의로 행 사이즈를 정해줌

array([[0, 1],
       [2, 3],
       [4, 5]])

In [29]:
## 참고 
a = np.array([[1,6],[2,7],[3,8],[4,9],[5,10]])
print(np.stack(a, axis = 0))
print(np.stack(a, axis = 1))## Transpose 한것처럼 쌓아줌  
print(a.T)

[[ 1  6]
 [ 2  7]
 [ 3  8]
 [ 4  9]
 [ 5 10]]
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]]
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]]


In [6]:
print(train_label)
print(test_label)
print(train_feature)
print(test_feature)

## Tensor("Cast_3:0", shape=(?, 10), dtype=float32) 
## shape는 배치사이즈가 처음에 들어가긴 하지만  총데이터 899개를 씀, 그이상으로 몇번 사용가능

Tensor("Cast:0", shape=(?, 1), dtype=float32)
Tensor("Cast_1:0", shape=(?, 1), dtype=float32)
Tensor("Cast_2:0", shape=(?, 10), dtype=float32)
Tensor("Cast_3:0", shape=(?, 10), dtype=float32)


In [7]:
## model 정하고 train 시키고 다음에 test 에 대입하는 함수 

def bin_model(x, activation =None, reuse = False):
    ## 모델 정하기(layer 정하기)
    layer1 = tf.layers.dense(x, units =5 , activation = activation, reuse = reuse, name = 'layer1') ## 
    layer2 = tf.layers.dense(layer1, units =6,activation = activation, reuse = reuse ,name = 'layer2')
    layer3 = tf.layers.dense(layer2, units =7,activation = activation, reuse = reuse,name = 'layer3' )
    layer4 = tf.layers.dense(layer3, units =8,activation = activation, reuse = reuse,name = 'layer4' )
    out = tf.layers.dense(layer4, units =1, reuse = reuse, name = 'out' ) 
    ## output은 activation 함수 사용하면 안됨 (loss를 linear로 구해야하기때문)
    ## unit =hidden layer의 크기 
    
## 활성함수 activation 함수는 시그모이드 함수를 잘 사용하지 않는다.
## 학습이 되지 않음 왜냐! 시그모이드의 기울기는 기점을 중심으로 퍼질때 0이 되기때문 
## loss 가 수렴하지 않아서 줄어들지 않음 
    
    print('layer1 shape {}'.format(layer1.shape))
    print('layer2 shape {}'.format(layer2.shape))
    print('layer3 shape {}'.format(layer3.shape))
    print('layer4 shape {}'.format(layer4.shape))
    print('out shape {}'.format(out.shape))
    
    return out 
    
    

train_out = bin_model(train_feature)
test_out = bin_model(test_feature,reuse = True) 
## train에서 업데이트 된 가중치 W와 b를 test에 대하여 다시 사용 = reuse 

for v in tf.trainable_variables():## train에서 업데이트 된 가중치 W와 b
    print(v)
    
    
saver = tf.train.Saver() ## save all Variables

layer1 shape (?, 5)
layer2 shape (?, 6)
layer3 shape (?, 7)
layer4 shape (?, 8)
out shape (?, 1)
layer1 shape (?, 5)
layer2 shape (?, 6)
layer3 shape (?, 7)
layer4 shape (?, 8)
out shape (?, 1)
<tf.Variable 'layer1/kernel:0' shape=(10, 5) dtype=float32_ref>
<tf.Variable 'layer1/bias:0' shape=(5,) dtype=float32_ref>
<tf.Variable 'layer2/kernel:0' shape=(5, 6) dtype=float32_ref>
<tf.Variable 'layer2/bias:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'layer3/kernel:0' shape=(6, 7) dtype=float32_ref>
<tf.Variable 'layer3/bias:0' shape=(7,) dtype=float32_ref>
<tf.Variable 'layer4/kernel:0' shape=(7, 8) dtype=float32_ref>
<tf.Variable 'layer4/bias:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'out/kernel:0' shape=(8, 1) dtype=float32_ref>
<tf.Variable 'out/bias:0' shape=(1,) dtype=float32_ref>


In [8]:
## loss 구하기 
loss = tf.losses.sigmoid_cross_entropy(train_label,train_out)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

tf.summary.scalar ('loss', loss) ## loss 에 대해서 summary collecting
merged = tf.summary.merge_all()

In [9]:
pred = tf.nn.sigmoid(test_out) ## prediction 
accuracy = tf.metrics.accuracy(test_label, tf.round(pred)) ## metrics = 검증 메트릭스

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    ## 세션을 초기화 하는 순간 변수 W에 그 값이 지정되는데
    ## 다음과 같이 변수들을 global_variables_initializer() 를 이용해서 초기화 한후, 초기화된 결과를 세션에 전달해 줘야 한다.

    sess.run(tf.local_variables_initializer())
    writer = tf.summary.FileWriter('data/',sess.graph)
    
    for i in range(1000):
        _,_loss, = sess.run([train_op,loss])
        _accuracy = sess.run(accuracy)
        _summ = sess.run(merged)
    
        _pred = sess.run(pred)
        writer.add_summary(_summ,i)
        
        if i %100 ==0:
            print('step:{} ,loss:{}, accuracy{}'.format(i, _loss, _accuracy))
    saver.save(sess,'data/saved parameters') 

step:0 ,loss:1.5148115158081055, accuracy(0.0, 0.5)
step:100 ,loss:0.0028549295384436846, accuracy(0.99000001, 0.99009901)
step:200 ,loss:0.0011999162379652262, accuracy(0.995, 0.99502486)
step:300 ,loss:0.0007296655094251037, accuracy(0.99666667, 0.99667776)
step:400 ,loss:0.0005145620089024305, accuracy(0.9975, 0.99750626)
step:500 ,loss:0.0003931887913495302, accuracy(0.99800003, 0.99800402)
step:600 ,loss:0.00031596189364790916, accuracy(0.99833333, 0.99833608)
step:700 ,loss:0.0002628252259455621, accuracy(0.99857146, 0.99857348)
step:800 ,loss:0.00022419585729949176, accuracy(0.99874997, 0.99875158)
step:900 ,loss:0.00019494109437800944, accuracy(0.99888891, 0.9988901)


In [11]:
## saver.save(sess,'data/saved parameters') 
## 세션에 돌아가는 그래프생성된 모든 정보들이 저장됨 
## 확장자 이름 없음 

In [ ]:
with tf.Session() as sess: ## saver로 지금 있는 모든 정보 저장
    saver.restore(sess, './data') ## 디렉토리만 넣어주면 됨 
    _pred = sess.run(pred)
    print(_pred)
    
## INFO:tensorflow:Restoring parameters from ./data 이라고 출력됨 

INFO:tensorflow:Restoring parameters from ./data


#### data 디렉토리로 tensorboard 로그연결
#### window powershell 열기 data가 있는 윗 디렉토리 Deeplearing_Tensorflow 로 들어가서 하기의 command 입력 

PS C:\Users\User\DS School Course Materials\Deeplearning_Tensorflow> tensorboard --logdir=data 입력 후 

http://localhost:6006 크롬에서 접속 

In [ ]:
## 궁금한점: placeholder, Variable 함수 사용안함 
# 그러나 tf.layers.dense 사용후 프린트 하면 나옴 ,, 왜그러지?  오토 레이어라고 함 